<a href="https://colab.research.google.com/github/Yoshida-2002/thesis/blob/main/CEG_omo_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/MyDrive/B4yoshida/NILE with pytorch lightning
%ls
%pwd

In [ ]:
!pip install transformers
!pip install pytorch-lightning

# data_loader

## data_reader.py

In [ ]:
# -*- coding: utf-8 -*-
# ========================================================
"""data_reader module is written for read files"""
# ========================================================


# ========================================================
# Imports
# ========================================================

import pandas as pd
import numpy as np


def read_csv(path: str) -> pd.DataFrame:
    """

    :param path:
    :return:
    """
    return pd.read_csv(path)

def read_tsv(path: str) -> pd.DataFrame:
    """

    :param path:
    :return:
    """
    return pd.read_csv(path, sep="\t", header=None)


def read_npy(path: str) -> np.ndarray:
    """
    load a list of numpy elements into memory
    :param path:
    :return:
    """
    return np.load(path, allow_pickle=True)


def read_excel(path:str) -> pd.DataFrame:
    """
    :param path: where to load data from
    :return: pd.DataFrame
    """
    return pd.read_excel(path, engine="openpyxl")



In [ ]:
train_path = '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/row/tsv/train2.tsv'
test_path = '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/row/tsv/test2.tsv'
val_path = '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/row/tsv/val2.tsv'

In [ ]:
train_data = read_tsv(train_path)
test_data = read_tsv(test_path)
val_data = read_tsv(val_path)

print(len(train_data))
print(len(test_data))
print(len(val_data))

In [ ]:
train_data

In [ ]:
print(train_data.isnull().values.sum())
print(test_data.isnull().values.sum())
print(val_data.isnull().values.sum())

In [ ]:
train_data.info()
test_data.info()
val_data.info()

## data_writer.py

In [ ]:
# -*- coding: utf-8 -*-
# ========================================================
"""data_writer module is written for write data in files"""
# ========================================================


# ========================================================
# Imports
# ========================================================
import pandas as pd
import numpy as np


def write_npy(path: str, data: list) -> None:
    """
    save a list of numpy elements into disk
    :param path:
    :param data:
    :return:
    """
    return np.save(path, data, allow_pickle=True)

def write_dataframe_in_tsv(data: pd.DataFrame, path: str) -> None:

  """
  save pd.Dataframe in tsv file
  :param path:
  :param data:

  """

  data.to_csv(path, sep='\t', index=False, header=False)



## data_missing_value_processor
- 欠損値の処理を行う
- やること
  1. 全ての値がNaNの列を削除する
  2. NaNを0で補完する

In [ ]:
def data_missing_value_processor(data: pd.DataFrame, path: str) -> pd.DataFrame:

  print('input data size : ' + str(len(data)))

  data_without_invalid_row = data.dropna(how="all")
  print(str(len(data) - len(data_without_invalid_row)) + " data was dropped because all of the values were NaN. ")

  processed_data = data_without_invalid_row.fillna(0)
  print("NaN in dataframe was filled with 0. ")
  print("processed input data size : " + str(len(processed_data)))
  print("removed data size : " + str(len(data) - len(processed_data)) )
  print('\n')

  write_dataframe_in_tsv(processed_data, path)

  return processed_data




In [ ]:

processed_train_data = data_missing_value_processor(
    train_data,
    '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/missing_value_processed/pocessed_train2.tsv'
)

processed_test_data = data_missing_value_processor(
    test_data,
    '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/missing_value_processed/pocessed_test2.tsv'
)

processed_val_data = data_missing_value_processor(
    val_data,
    '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/missing_value_processed/pocessed_val2.tsv'
)

In [ ]:
# processed_train_data

In [ ]:
# processed_train_data[4].replace(0, 'None', inplace=True)
# processed_train_data[5].replace(0, 'None', inplace=True)
# processed_train_data[6].replace(0, 'None', inplace=True)
# processed_train_data[7].replace(0, 'None', inplace=True)
# processed_train_data[8].replace(0, 'None', inplace=True)
# processed_train_data[14].replace(0, 'None', inplace=True)
# processed_train_data[15].replace(0, 'None', inplace=True)

In [ ]:
# processed_train_data.head()

In [ ]:
# print( (processed_train_data == 0).sum())

In [ ]:
# processed_train_data = processed_train_data.replace(
#     {'ture'       :torch.tensor([1,0]),
#      'mostly-true':torch.tensor([1,0]),
#      'half-true'  :torch.tensor([1,0]),
#      'barely-true':torch.tensor([0,1]),
#      'false'      :torch.tensor([0,1]),
#      'pants-fire' :torch.tensor([0,1])})

In [ ]:
# print(processed_train_data[4]+ ' ' + processed_train_data[5] + ' ' + processed_train_data[6] + ' ' + processed_train_data[7] + ' ' + processed_train_data[8] + ' ' + processed_train_data[14])

##data_pre_processor_for_LIAR_PLUS_Dataset
- LIAR_PLUS_Datasetクラスの入力に用いるdataframeを作成する
- 作成したdataframeを保存する
- statement, metadata, justification, credit_scoreの四つの列からなるdataframeを出力する
- やること
  1. 値として0(元々は欠損値)を持つ場合、"None"で補完する
  2. 列を結合してstaetment, metadata, justification, credit_scoreを作成する
  3. credit_scoreのnanを0.5で補完する
  4. 作成したdataframeを保存する


In [ ]:
def data_pre_processor_for_LIAR_PLUS_Dataset(data: pd.DataFrame, path: str) -> pd.DataFrame:

  # 1. 値として0(元々は欠損値)を持つ場合、"None"で補完する

  data[4].replace(0, 'None', inplace=True)
  data[5].replace(0, 'None', inplace=True)
  data[6].replace(0, 'None', inplace=True)
  data[7].replace(0, 'None', inplace=True)
  data[8].replace(0, 'None', inplace=True)
  data[14].replace(0, 'None', inplace=True)
  data[15].replace(0, 'None', inplace=True)

  # 2. 列を結合してstaetment, metadata, justificationを作成する

  data_for_LIAR_PLUS_Dataset = pd.DataFrame(
      data = {
      'id' : data[0],
      'label' : data[2],
      'statement' : data[3],
      'metadata' : data[4]+ ' ' + data[5] + ' ' + data[6] + ' ' + data[7] + ' ' + data[8] + ' ' + data[14],
      'justification' : data[15],
      'credit_score' : (data[12]*0.2 + data[11]*0.5 + data[9]*0.75 + data[10]*0.9 + data[13]*1)/(data[12] + data[11] + data[9] + data[10] + data[13]),
      'unique_id' : data[1]
      })

  data_for_LIAR_PLUS_Dataset['credit_score'] = data_for_LIAR_PLUS_Dataset['credit_score'].fillna(0.5)

  write_dataframe_in_tsv(data_for_LIAR_PLUS_Dataset, path)

  print("dataframe was saved in" + path + "\n")

  data_for_LIAR_PLUS_Dataset.info()


  return data_for_LIAR_PLUS_Dataset

In [ ]:
train_data_for_LIAR_PLUS_Dataset = data_pre_processor_for_LIAR_PLUS_Dataset(
    processed_train_data,
    '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/input_for_lightning_model/main/input_train2.tsv'
    )
test_data_for_LIAR_PLUS_Dataset = data_pre_processor_for_LIAR_PLUS_Dataset(
    processed_test_data,
    '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/input_for_lightning_model/main/input_test2.tsv'
    )
val_data_for_LIAR_PLUS_Dataset = data_pre_processor_for_LIAR_PLUS_Dataset(
    processed_val_data,
    '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/input_for_lightning_model/main/input_val2.tsv'
    )

In [ ]:
write_dataframe_in_tsv(
    train_data_for_LIAR_PLUS_Dataset[:120],
    '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/input_for_lightning_model/toy/toy_input_train2.tsv')
write_dataframe_in_tsv(
    test_data_for_LIAR_PLUS_Dataset[:40],
    '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/input_for_lightning_model/toy/toy_input_test2.tsv')
write_dataframe_in_tsv(
    val_data_for_LIAR_PLUS_Dataset[:40],
    '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/input_for_lightning_model/toy/toy_input_val2.tsv')

In [ ]:
base_train_data = train_data_for_LIAR_PLUS_Dataset
base_train_data

# prompt_generator
・プロンプトの作成を一任


In [ ]:
def prompt_generator(data: pd.DataFrame, path :str) -> pd.DataFrame :
  """
  this function is for generating and saving prompts

  param  : stage("fine-tuning" of "completion")
  param  : data(row data for prompt generation)
  param  : path(path to save generated promps dataframe)
  return : prompts(generated prompts)
  """
  generated_prompts = "Statement: " + data[2] + " Metadata: " + data[3]

  prompts_df = pd.DataFrame(
      data = {
      'id' : data[0],
      'label' : data[1],
      'concat' : generated_prompts,
      'justification': data[4],
      'credit_score' : data[5],
      'unique_id' : data[6]
      })

  write_dataframe_in_tsv(prompts_df, path)

  return prompts_df


In [ ]:
train_prompt_path = "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts/main/train_prompts.tsv"
test_prompt_path = "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts/main/test_prompts.tsv"
val_prompt_path = "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts/main/val_prompts.tsv"

train_prompts = read_tsv("/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/input_for_lightning_model/main/input_train2.tsv")
test_prompts = read_tsv("/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/input_for_lightning_model/main/input_test2.tsv")
val_prompts = read_tsv("/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/processed/input_for_lightning_model/main/input_val2.tsv")

train_prompts = prompt_generator(train_prompts, train_prompt_path)
test_prompts = prompt_generator(test_prompts, test_prompt_path)
val_prompts = prompt_generator(val_prompts, val_prompt_path)

print(len(train_prompts))
train_prompts

In [ ]:
x = read_tsv(train_prompt_path)
x.info()

# generated_prompts_separator
- ラベルごとに分類する

In [ ]:
def generated_prompts_separator(path: dict, data: pd.DataFrame) -> dict:

  true_set = pd.DataFrame()
  false_set = pd.DataFrame()
  total_len = len(data)

  for i in range(len(data)):

    data_row = data.iloc[i]
   # print(data_row)

    if data_row[1] == "true":
        true_set = true_set.append(data_row, ignore_index=True)
    elif data_row[1] == "half-true":
        true_set = true_set.append(data_row, ignore_index=True)
    elif data_row[1] == "mostly-true":
        true_set = true_set.append(data_row, ignore_index=True)
    elif data_row[1] == "false":
        false_set = false_set.append(data_row, ignore_index=True)
    elif data_row[1] == "barely-true":
        false_set = false_set.append(data_row, ignore_index=True)
    elif data_row[1] == "pants-fire":
        false_set = false_set.append(data_row, ignore_index=True)
    else:
        print(f"Invalid label was found! {data_row[1]}")

  print("===================================================\n")
  print(f"total len : {total_len}")
  print(f"true_set : {len(true_set)}")
  print(f"false_set : {len(false_set)}")
  print("===================================================\n")

  write_dataframe_in_tsv(true_set, path["true_set"])
  write_dataframe_in_tsv(false_set, path["false_set"])


  return dict(
      true_set=true_set,
      false_set=true_set
  )


In [ ]:
fine_main_train_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/train/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/train/false_sets.tsv"
}

fine_main_test_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/test/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/test/false_sets.tsv"
}

fine_main_val_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/val/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/val/false_sets.tsv"
}

fine_toy_train_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/train/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/train/false_sets.tsv"
}

fine_toy_test_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/test/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/test/false_sets.tsv"
}

fine_toy_val_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/val/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/val/false_sets.tsv"
}

In [ ]:
# main_train_df = read_tsv(train_prompt_path)
# main_test_df = read_tsv(test_prompt_path)
# main_val_df = read_tsv(val_prompt_path)

# toy_train_df = main_train_df[:600]
# toy_test_df = main_test_df[:200]
# toy_val_df = main_val_df[:200]


# main_train_set = generated_prompts_separator(fine_main_train_path_dict, main_train_df)
# main_test_set = generated_prompts_separator(fine_main_test_path_dict, main_test_df)
# main_val_set = generated_prompts_separator(fine_main_val_path_dict, main_val_df)

# toy_train_set = generated_prompts_separator(fine_toy_train_path_dict, toy_train_df)
# toy_test_set = generated_prompts_separator(fine_toy_test_path_dict, toy_test_df)
# toy_val_set = generated_prompts_separator(fine_toy_val_path_dict, toy_val_df)


In [ ]:
y = read_tsv("/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/train/false_sets.tsv")
y.info()

# Google Driveへ接続しディレクトリを移動

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/B4yoshida/NILE with pytorch lightning
%ls
%pwd

/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning
' CEG omo on colab'  'data for CEG'     models             model_transformers/
 data/                lightning_logs/  'models のコピー'   src/


'/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning'

# TransformersとPytorch Lightningをインストール

In [ ]:
!pip install transformers
!pip install pytorch-lightning

In [ ]:
import transformers
import pytorch_lightning as pl
import pandas as pd
print(transformers.__version__)
print(pl.__version__)

# Pathの定義

In [ ]:
fine_main_train_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/train/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/train/false_sets.tsv"
}

fine_main_test_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/test/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/test/false_sets.tsv"
}

fine_main_val_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/val/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/main/val/false_sets.tsv"
}

fine_toy_train_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/train/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/train/false_sets.tsv"
}

fine_toy_test_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/test/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/test/false_sets.tsv"
}

fine_toy_val_path_dict = {
    "true_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/val/true_sets.tsv",
    "false_set" : "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/toy/val/false_sets.tsv"
}


train_data_path = "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/fine-tuning/toy/train/true_sets.tsv"
test_data_path = "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/fine-tuning/toy/test/true_sets.tsv"
val_data_path = "/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/data/prompts_set/fine-tuning/toy/val/true_sets.tsv"

def read_tsv(path: str) -> pd.DataFrame:
    """

    :param path:
    :return:
    """
    return pd.read_csv(path, sep="\t", header=None)


# class LIAR_PLUS_Dataset_For_CEG(Dataset):を定義する

In [ ]:
# -*- coding: utf-8 -*-
# pylint: disable-msg=import-error
# pylint: disable-msg=no-member
# ========================================================
"""dataset module is written for create data module"""
# ========================================================


# ========================================================
# Imports
# ========================================================
import pandas as pd
import pytorch_lightning as pl
import torch
import time

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

from typing import Optional

class LIAR_PLUS_Dataset_For_CEG(Dataset):
    """
    this class is for encoding input dataframe for GPT2

    Input dataframe needs to be consist of 5 columns, "id", "label", "prompt", "justification" and "credit_score"

    later we wrap a lightning data module around it.
    """

    def __init__(self, data: pd.DataFrame, config_dict: dict):
      # max_token_length should be flexible depending on input sequence so it needs to be changed later.
        self.data = data
        self.config_dict = config_dict
        self.prepare_tokenizer = Prepare_Tokenizer(self.config_dict)
        self.tokenizer = self.prepare_tokenizer.get_tokenizer()
        self.prepare_tokenizer.get_tokenizer_info()
        self.max_token_len = config_dict['MAX_LENGTH']
        # print("Before")
        # print(self.tokenizer.all_special_tokens)
        # print(self.tokenizer.all_special_ids)
        # special_tokens_dict = {
        # 'additional_special_tokens': ['[EXP]'],
        # "eos_token" : "<|endoftext|>",
        # "pad_token" : "<|endoftext|>"
        # }
        # self.tokenizer.add_special_tokens(special_tokens_dict)
        # self.exp_id = tokenizer.convert_tokens_to_ids('[EXP]')
        # self.eos_id = tokenizer.convert_tokens_to_ids('<|endoftext|>')
        # self.pad_id = tokenizer.convert_tokens_to_ids('<|endoftext|>')
        # print("After")
        # print(tokenizer.all_special_tokens)
        # print(tokenizer.all_special_ids)
        # print(f"exp_id : {self.exp_id}")
        # print(f"eos_id : {self.eos_id}")
        # print(f"pad_id : {self.pad_id}\n")
        # print(f"len(tokenizer) : {len(self.tokenizer)}")

         # it could be 512

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):

        data_row = self.data.iloc[index]
        concat_text = data_row[2] + "[EXP]"
        justification_text = data_row[3]
        full_prompt_text = concat_text + justification_text + "[EOS]"

        concat_text_length = len(self.tokenizer.tokenize(concat_text))
        # bartではspecial_token<s>が文頭に付くため、その分だけ入力長が1大きくなる
        if self.config_dict["MODEL_NAME"] == 'facebook/bart-base':
            concat_text_length += concat_text_length

        if concat_text_length > self.max_token_len:
            concat_text_length = self.max_token_len
        justification_text_length = len(self.tokenizer.tokenize(justification_text))
        if justification_text_length > self.max_token_len:
            justification_text_length = self.max_token_len
        full_prompt_text_length = len(self.tokenizer.tokenize(full_prompt_text))
        if full_prompt_text_length > self.max_token_len:
            full_prompt_text_length = self.max_token_len

        if data_row[1] == "true":
            label = torch.tensor([1,0], dtype=torch.float32)
        elif data_row[1] == "half-true":
            label = torch.tensor([1,0], dtype=torch.float32)
        elif data_row[1] == "mostly-true":
            label = torch.tensor([1,0], dtype=torch.float32)
        elif data_row[1] == "false":
            label = torch.tensor([0,1], dtype=torch.float32)
        elif data_row[1] == "barely-true":
            label = torch.tensor([0,1], dtype=torch.float32)
        elif data_row[1] == "pants-fire":
            label = torch.tensor([0,1], dtype=torch.float32)
        else:
            print(f"Invalid label was found! {data_row[1]}")
            time.sleep(30)

        concat_encoding = self.tokenizer.encode_plus(
            concat_text,
            add_special_tokens=True,
            max_length = self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        full_prompt_encoding = self.tokenizer.encode_plus(
            full_prompt_text,
            add_special_tokens=True,
            max_length = self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        # labels = full_prompt_encoding["input_ids"].flatten()
        # labels[:concat_text_length] = -100 # cross_entropy_ignore_index

        # print("=================================================================\n")
        # print(f"concat_text : {concat_text}\n")
        # print(f"concat_text_length : {concat_text_length}\n")
        # print(f"justification_text : {justification_text}\n")
        # print(f"justification_text_length : {justification_text_length}\n")
        # print(f"full_prompt_text : {full_prompt_text}\n")
        # print(f"full_prompt_text_length : {full_prompt_text_length}\n")
        # print(f"labels before padding : {labels}\n")
        # print(f"labels after padding: {labels}\n")
        # print("=================================================================\n")

        return dict(
            input_ids1=concat_encoding["input_ids"].flatten(),
            attention_mask1=concat_encoding["attention_mask"].flatten(),
            input_ids2=full_prompt_encoding["input_ids"].flatten(),
            attention_mask2=full_prompt_encoding["attention_mask"].flatten(),
            concat_text_length=concat_text_length,
            full_prompt_text_length=full_prompt_text_length,
            # labels = labels
        )


# class LIAR_PLUS_DataModule_For_CEG(pl.LightningDataModule)を定義する

In [ ]:
class LIAR_PLUS_DataModule_For_CEG(pl.LightningDataModule):

    def __init__(self, config_dict, train_df: pd.DataFrame, test_df: pd.DataFrame, val_df: pd.DataFrame):
        super().__init__()
        self.config_dict = config_dict
        self.train_df = train_df
        self.test_df = test_df
        self.val_df = val_df
        self.training_batch_size = config_dict["TRAINING_BATCH_SIZE"]
        self.validation_batch_size = config_dict["VALIDATION_BATCH_SIZE"]
        self.test_batch_size = config_dict["TEST_BATCH_SIZE"]
        self.max_token_len = self.config_dict["MAX_LENGTH"]
        self.train_dataset, self.test_dataset, self.val_dataset = None, None, None

        # 以下の処理はDataset内でやってくれるのかも
        # print("Before")
        # print(self.tokenizer.all_special_tokens)
        # print(self.tokenizer.all_special_ids)
        # special_tokens_dict = {'additional_special_tokens': ['[EXP]'], "eos_token" : "[EOS]"}
        # self.tokenizer.add_special_tokens(special_tokens_dict)
        # self.exp_id = tokenizer.convert_tokens_to_ids('[EXP]')
        # self.eos_id = tokenizer.convert_tokens_to_ids('[EOS]')
        # print("After")
        # print(tokenizer.all_special_tokens)
        # print(tokenizer.all_special_ids)
        # print(f"exp_id : {self.exp_id}")
        # print(f"eos_id : {self.eos_id}\n")


    def setup(self, stage: Optional[str] = None) -> None:
        self.train_dataset = LIAR_PLUS_Dataset_For_CEG(self.train_df, self.config_dict)
        print(f"train_dataset size : {len(self.train_dataset)}\n")
        self.test_dataset = LIAR_PLUS_Dataset_For_CEG(self.test_df, self.config_dict)
        print(f"test_dataset size : {len(self.test_dataset)}\n")
        self.val_dataset = LIAR_PLUS_Dataset_For_CEG(self.val_df, self.config_dict)
        print(f"val_dataset size : {len(self.val_dataset)}\n")


    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.training_batch_size,
            shuffle=True,
            drop_last=True,
            num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.validation_batch_size,
            drop_last=True,
            num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.test_batch_size,
            drop_last=True,
            num_workers=2
        )

# helperを定義する

In [ ]:
# -*- coding: utf-8 -*-
# ========================================================
"""helper module is written for write useful function in indexer package"""
# ========================================================


# ========================================================
# Imports
# ========================================================
from pytorch_lightning.callbacks import ModelCheckpoint


def build_checkpoint_callback(config_dict, filename="QTag-{epoch:02d}-{val_loss:.2f}",
                              monitor="val_loss"):
    """

    :param save_top_k:
    :param filename:
    :param monitor:
    :return:
    """
    # saves a file like: input/QTag-epoch=02-val_loss=0.32.ckpt
    checkpoint_callback = ModelCheckpoint(monitor=monitor,  # monitored quantity
                                          filename=filename,
                                          save_top_k=config_dict['SAVE_TOP_K'],  # save the top k models
                                          dirpath=config_dict['SAVED_MODEL_PATH'],
                                          mode="min",  # mode of the monitored quantity for optimization
                                          )
    print(f"checkpoint_callback : {checkpoint_callback}")
    return checkpoint_callback

# class CEG(pl.LightningModule)を定義する

In [ ]:
# -*- coding: utf-8 -*-
# pylint: disable=too-many-arguments
# pylint: disable=import-error
# ========================================================
"""This module is written for write BERT classifier."""
# ========================================================


# ========================================================
# Imports
# ========================================================
from typing import List
import pytorch_lightning as pl
from torch import nn
import torch
import torchmetrics
from transformers import GPT2Tokenizer, AutoModel, AutoConfig, AdamW, get_linear_schedule_with_warmup, GPT2LMHeadModel, T5ForConditionalGeneration, BartForConditionalGeneration

class CEG(pl.LightningModule):
    """
    creates a pytorch lightning model
    """

    def __init__(self, config_dict,
                 n_warmup_steps: int = None,
                 n_training_steps: int = None,
                 n_classes: int = None):
        super().__init__()
        self.config_dict = config_dict
        self.prepare_tokenizer = Prepare_Tokenizer(self.config_dict)
        self.tokenizer = self.prepare_tokenizer.get_tokenizer()
        self.prepare_tokenizer.get_tokenizer_info()
        self.prepare_model = Prepare_Model(config=self.config_dict, prepare_tokenizer=self.prepare_tokenizer)
        self.model = self.prepare_model.get_model()
        self.prepare_model.get_model_info()
        # self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

        # print("Before")
        # print(self.tokenizer.all_special_tokens)
        # print(self.tokenizer.all_special_ids)
        # special_tokens_dict = {
        #     'additional_special_tokens': ['[EXP]'],
        #     "eos_token" : "<|endoftext|>",
        #     "pad_token" : "<|endoftext|>"
        #     }
        # self.tokenizer.add_special_tokens(special_tokens_dict)
        # exp_id = self.tokenizer.convert_tokens_to_ids('[EXP]')
        # eos_id = self.tokenizer.convert_tokens_to_ids('<|endoftext|>')
        # pad_id = self.tokenizer.convert_tokens_to_ids('<|endoftext|>')
        # print("After")
        # print(self.tokenizer.all_special_tokens)
        # print(self.tokenizer.all_special_ids)
        # print(f"exp_id : {exp_id}")
        # print(f"eos_id : {eos_id}")
        # print(f"pad_id : {pad_id}\n")
        # print(f"len(self.tokenizer) : {len(self.tokenizer)}")

        # gpt2config = AutoConfig.from_pretrained('gpt2', bos_token_id=self.tokenizer.bos_token_id, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id, additional_special_tokens_id = exp_id, output_hidden_states=False)

        # self.model = GPT2LMHeadModel.from_pretrained("gpt2", config=gpt2config)
        # self.model.resize_token_embeddings(len(self.tokenizer))
        # self.lm_head = self.model.lm_head
        # new_weights = torch.cat([self.lm_head.weight[:-1, :], torch.zeros(1, self.lm_head.weight.shape[1]) -10000])
        # self.model.lm_head.weight = torch.nn.Parameter(new_weights)
        self.softmax = nn.Softmax(dim=1)
        self.batch_size = config_dict['TRAINING_BATCH_SIZE']
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.CrossEntropyLoss(ignore_index=self.config_dict['CROSS_ENTROPY_IGNORE_INDEX'])
        self.save_hyperparameters()

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2, concat_text_length, full_prompt_text_length):

        # print("===============================forward=================================\n")
        labels = input_ids2
        # print(f"labels : {labels}")
        # print(f"len(labels) : {len(labels)}")
        print(full_prompt_text_length)
        batch_max_length = max(full_prompt_text_length)
        print(f"batch_max_length : {batch_max_length}")


        for i in range(len(labels)):
            labels[i][:concat_text_length[i]] = -100 # cross_entropy_ignore_index
            labels[i][full_prompt_text_length[i]:] = -100
        print(f"input_ids1 before reshaping : {input_ids1.shape}")
        print(f"labels before reshaping : {labels.shape}")
        print(f"attention_mask1 before reshaping : {attention_mask1.shape}")
        labels = labels[:,:batch_max_length]
        input_ids1 = input_ids1[:,:batch_max_length]
        attention_mask1 = attention_mask1[:,:batch_max_length]
        print(f"input_ids1 after reshaping : {input_ids1.shape}")
        print(f"labels after reshaping : {labels.shape}")
        print(f"attention_mask1 after reshaping : {attention_mask1.shape}")
            # print(f"labels after -100 padding : {labels[0]}")
        # print(f"len(labels[0]) before shifting : {len(labels[0])}\n")
        # print(f"input_ids1 : {input_ids1}\n")
        # print(f"attention_mask1 : {attention_mask1}\n")
        # print(f"input_ids2 : {input_ids2}\n")
        # print(f"concat_text_length : {concat_text_length}\n")
        # print(f"full_prompt_length : {full_prompt_text_length}\n")
        # print(f"labels : {labels}\n")
        # print(f"labels before shifting : {labels}, labels.size() : {labels.size()}\n")
        # labels = labels[0][1:]
        # labels = torch.Tensor(labels)
        # print(f"labels after shifting : {labels}, labels.size() : {labels.size()}\n")
        # additional_pad = tokenizer.tokenize("[PAD]")
        # additional_pad = torch.Tensor(tokenizer.convert_tokens_to_ids(additional_pad))
        # additional_pad = torch.Tensor(additional_pad)
        # labels = torch.cat((labels, additional_pad))
        # print(f"labels after shifting : {labels}, labels.size() : {labels.size()}\n")

        #outputs = self.model(input_ids=input_ids1, attention_mask=attention_mask1, labels=labels)
        outputs = self.model(input_ids=input_ids1, attention_mask=attention_mask1, labels=labels)
        logits = outputs.logits
        loss = 0

        loss = outputs.loss

        # print(f"outputs : {outputs}\n")
        # print(f"outputs type : {type(outputs)}")
        # print(f"input.shape : {outputs.shape}")
        # print(f"logits : {logits}")
        # print(f"logits type : {type(logits)}")
        # print(f"logits shape : {logits.shape}")
        # print(f"loss : {loss}")
        softmax_output = self.softmax(logits)
        arg = torch.argmax(softmax_output, dim=2)
        # print(f"argmax(ouput) : {arg}") # --> tensor
        # print(f"argmax(ouput).size() : {arg.size()}")
        # print(f"argmax(output).dtype() : {arg.dtype}")
        # print(arg)
        generated = self.tokenizer.decode(arg[0])
        print(f"generated text: {generated}\n")
        print(f"len(generated) : {len(self.tokenizer.tokenize(generated))}\n")
        # print(f"loss type : {type(loss)}\n")
        # print(f"loss shape : {loss.shape}\n")

        return loss, outputs

    def training_step(self, batch, batch_idx):
        """

        :param batch:
        :param batch_idx:
        :return:
        """
        # print(f"training batch : {batch}")
        input_ids1 = batch["input_ids1"]
        attention_mask1 = batch["attention_mask1"]
        input_ids2 = batch["input_ids2"]
        attention_mask2 = batch["attention_mask2"]
        concat_text_length = batch["concat_text_length"]
        full_prompt_text_length = batch["full_prompt_text_length"]
        # labels = batch["labels"]

        loss, output = self(input_ids1, attention_mask1, input_ids2, attention_mask2, concat_text_length, full_prompt_text_length)


        self.log("train_loss", loss, prog_bar=True, logger=True)
        print("===============================training=================================\n")
        # print(f"output : {outputs}\n")
        # print(f"output type : {type(outputs)}\n")
        print(f"loss : {loss}\n")
        # print(f"loss type : {type(loss)}\n")
        # print(f"loss shape : {loss.shape}\n")

        return {"loss": loss, "predictions": output}

    def validation_step(self, batch, batch_idx):
        """

        :param batch:
        :param batch_idx:
        :return:
        """
        # print(f"validation batch : {batch}")
        input_ids1 = batch["input_ids1"]
        attention_mask1 = batch["attention_mask1"]
        input_ids2 = batch["input_ids2"]
        attention_mask2 = batch["attention_mask2"]
        concat_text_length = batch["concat_text_length"]
        full_prompt_text_length = batch["full_prompt_text_length"]
        # labels = batch["labels"]

        loss, output = self(input_ids1, attention_mask1, input_ids2, attention_mask2, concat_text_length, full_prompt_text_length)

        self.log("val_loss", loss, prog_bar=True, logger=True, on_step=False)

        print("===============================validating=================================\n")
        # print(f"output : {outputs}\n")
        # print(f"output type : {type(outputs)}\n")
        print(f"loss : {loss}\n")
        # print(f"loss type : {type(loss)}\n")
        # print(f"loss shape : {loss.shape}\n")

        return loss

    def test_step(self, batch, batch_idx):
        """

        :param batch:
        :param batch_idx:
        :return:
        """

        print("test batch : {batch}")
        input_ids1 = batch["input_ids1"]
        attention_mask1 = batch["attention_mask1"]
        input_ids2 = batch["input_ids2"]
        attention_mask2 = batch["attention_mask2"]
        concat_text_length = batch["concat_text_length"]
        full_prompt_text_length = batch["full_prompt_text_length"]
        # labels = batch["labels"]

        loss, output = self(input_ids1, attention_mask1, input_ids2, attention_mask2, concat_text_length, full_prompt_text_length)

        self.log("test_loss", loss, prog_bar=True, logger=True)

        print("===============================testing=================================\n")
        # print(f"output : {outputs}\n")
        # print(f"output type : {type(outputs)}\n")
        print(f"loss : {loss}\n")
        # print(f"loss type : {type(loss)}\n")
        # print(f"loss shape : {loss.shape}\n")

        return loss

    def configure_optimizers(self):
        """

        :return:
        """
        optimizer = AdamW(self.parameters(), lr=self.config_dict['LR'])
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.n_warmup_steps,
                                                    num_training_steps=self.n_training_steps)
        return dict(optimizer=optimizer, lr_scheduler=dict(scheduler=scheduler, interval="step"))

# calculate_warmup_stepsを定義する

In [ ]:
# -*- coding: utf-8 -*-
# ========================================================
"""This module is written for write useful function."""
# ========================================================


# ========================================================
# Imports
# ========================================================

import pandas as pd

def calculate_warmup_steps(train_df: pd.DataFrame, num_epochs: int, batch_size: int):
    steps_per_epoch = len(train_df) // batch_size
    total_training_steps = steps_per_epoch * num_epochs
    warmup_steps = total_training_steps // 5
    return total_training_steps, warmup_steps

# ModelConfigを定義する
- colab上では辞書
- モジュール分けするときはjsonに

## GPT2

In [ ]:
# GPT2-small
gpt2_small_config = {
    'MODEL_NAME' : 'gpt2',
    'TOKENIZER_NAME' : 'gpt2',
    'MAX_LENGTH' : 512,
    'TRAINING_BATCH_SIZE' : 16,
    'VALIDATION_BATCH_SIZE' : 16,
    'TEST_BATCH_SIZE' : 1,
    'TRAINING_EPOCHS' : 50,
    'SAVE_TOP_K' : 1,
    'SAVED_MODEL_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/src/models',
    'CSV_LOGGER_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/weights/csv_log',
    'TRAINING_DATA_PATH' : fine_toy_train_path_dict["true_set"],
    'VALIDATION_DATA_PATH' : fine_toy_val_path_dict["true_set"],
    'TEST_DATA_PATH' : fine_toy_test_path_dict["true_set"],
    'LR' : 7e-5,
    'RANDOM_SEED' : 42,
    'CROSS_ENTROPY_IGNORE_INDEX' : -100
}

# GPT2-medium
gpt2_medium_config = {
    'MODEL_NAME' : 'gpt2-medium',
    'TOKENIZER_NAME' : 'gpt2-medium',
    'MAX_LENGTH' : 512,
    'TRAINING_BATCH_SIZE' : 16,
    'VALIDATION_BATCH_SIZE' : 16,
    'TEST_BATCH_SIZE' : 1,
    'TRAINING_EPOCHS' : 50,
    'SAVE_TOP_K' : 1,
    'SAVED_MODEL_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/src/models',
    'CSV_LOGGER_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/weights/csv_log',
    'TRAINING_DATA_PATH' : fine_toy_train_path_dict["true_set"],
    'VALIDATION_DATA_PATH' : fine_toy_val_path_dict["true_set"],
    'TEST_DATA_PATH' : fine_toy_test_path_dict["true_set"],
    'LR' : 7e-5,
    'RANDOM_SEED' : 42,
    'CROSS_ENTROPY_IGNORE_INDEX' : -100
}

## T5

In [ ]:
# T5-small
t5_small_config = {
    'MODEL_NAME' : 't5-small',
    'TOKENIZER_NAME' : 't5-small',
    'MAX_LENGTH' : 512,
    'TRAINING_BATCH_SIZE' : 16,
    'VALIDATION_BATCH_SIZE' : 16,
    'TEST_BATCH_SIZE' : 1,
    'TRAINING_EPOCHS' : 50,
    'SAVE_TOP_K' : 1,
    'SAVED_MODEL_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/src/models',
    'CSV_LOGGER_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/weights/csv_log',
    'TRAINING_DATA_PATH' : fine_toy_train_path_dict["true_set"],
    'VALIDATION_DATA_PATH' : fine_toy_val_path_dict["true_set"],
    'TEST_DATA_PATH' : fine_toy_test_path_dict["true_set"],
    'LR' : 7e-5,
    'RANDOM_SEED' : 42,
    'CROSS_ENTROPY_IGNORE_INDEX' : -100
}

# T5-base
t5_base_config = {
    'MODEL_NAME' : 't5-base',
    'TOKENIZER_NAME' : 't5-base',
    'MAX_LENGTH' : 512,
    'TRAINING_BATCH_SIZE' : 16,
    'VALIDATION_BATCH_SIZE' : 16,
    'TEST_BATCH_SIZE' : 1,
    'TRAINING_EPOCHS' : 50,
    'SAVE_TOP_K' : 1,
    'SAVED_MODEL_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/src/models',
    'CSV_LOGGER_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/weights/csv_log',
    'TRAINING_DATA_PATH' : fine_toy_train_path_dict["true_set"],
    'VALIDATION_DATA_PATH' : fine_toy_val_path_dict["true_set"],
    'TEST_DATA_PATH' : fine_toy_test_path_dict["true_set"],
    'LR' : 7e-5,
    'RANDOM_SEED' : 42,
    'CROSS_ENTROPY_IGNORE_INDEX' : -100
}

## BART

In [ ]:
# BART_base
bart_base_config = {
    'MODEL_NAME' : 'facebook/bart-base',
    'TOKENIZER_NAME' : 'facebook/bart-base',
    'MAX_LENGTH' : 512,
    'TRAINING_BATCH_SIZE' : 16,
    'VALIDATION_BATCH_SIZE' : 16,
    'TEST_BATCH_SIZE' : 1,
    'TRAINING_EPOCHS' : 50,
    'SAVE_TOP_K' : 1,
    'SAVED_MODEL_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/src/models',
    'CSV_LOGGER_PATH' : '/content/drive/MyDrive/B4yoshida/NILE with pytorch lightning/weights/csv_log',
    'TRAINING_DATA_PATH' : fine_toy_train_path_dict["true_set"],
    'VALIDATION_DATA_PATH' : fine_toy_val_path_dict["true_set"],
    'TEST_DATA_PATH' : fine_toy_test_path_dict["true_set"],
    'LR' : 7e-5,
    'RANDOM_SEED' : 42,
    'CROSS_ENTROPY_IGNORE_INDEX' : -100
}

# class Prepare_tokenizerを定義する

In [ ]:
from transformers import AutoTokenizer
import warnings

class Prepare_Tokenizer():
    def __init__(self, config:dict):
        self.config = config
        self.TOKENIZER_NAME = self.config['TOKENIZER_NAME']
        self.tokenizer = None
        self.tokenizer_length = None
        self.additional_special_tokens = None
        self.eos_token = None
        self.eos_token_id = None
        self.pad_token = None
        self.pad_token_id = None
        self.exp_token = '[EXP]'
        self.exp_token_id = None
        self.initialization_flag = 0

    def get_tokenizer(self):
        if self.initialization_flag != 0:
            raise ValueError(f"tokenizer is already initialized. This instance is for {self.tokenizer}")

        if self.TOKENIZER_NAME in ['gpt2', 'gpt2-medium']:
            self.initialization_flag = 1
            self.tokenizer = AutoTokenizer.from_pretrained(self.TOKENIZER_NAME)
            print(f"tokenizer : AutoTokenizer.from_pretrained({self.TOKENIZER_NAME})")
            print("Before adding additional_special_tokens")
            print(self.tokenizer.all_special_tokens)
            print(self.tokenizer.all_special_ids)
            special_tokens_dict = {
            'additional_special_tokens': ['[EXP]'],
            "eos_token" : "<|endoftext|>",
            "pad_token" : "<|endoftext|>"
            }
            self.eos_token = "<|endoftext|>"
            self.pad_token = "<|endoftext|>"
            self.tokenizer.add_special_tokens(special_tokens_dict)
            self.exp_token_id = self.tokenizer.convert_tokens_to_ids('[EXP]')
            self.eos_token_id = self.tokenizer.convert_tokens_to_ids('<|endoftext|>')
            self.pad_token_id = self.tokenizer.convert_tokens_to_ids('<|endoftext|>')
            print("After adding additional_special_tokens")
            print(self.tokenizer.all_special_tokens)
            print(self.tokenizer.all_special_ids)
            print(f"exp_id : {self.exp_token_id}")
            print(f"eos_id : {self.eos_token_id}")
            print(f"pad_id : {self.pad_token_id}")
            self.tokenizer_length = len(self.tokenizer)
            print(f"len(tokenizer) : {self.tokenizer_length}")

            return self.tokenizer

        elif self.TOKENIZER_NAME in ['t5-small','t5-base']:
            self.initialization_flag = 1
            self.tokenizer = AutoTokenizer.from_pretrained(self.TOKENIZER_NAME)
            print(f"tokenizer : AutoTokenizer.from_pretrained({self.TOKENIZER_NAME})")
            print("Before adding additional_special_tokens")
            print(self.tokenizer.all_special_tokens)
            print(self.tokenizer.all_special_ids)
            special_tokens_dict = {
            'additional_special_tokens': ['[EXP]'],
            }
            self.eos_token = '</s>'
            self.pad_token = '<pad>'
            self.tokenizer.add_special_tokens(special_tokens_dict)
            self.exp_token_id = self.tokenizer.convert_tokens_to_ids('[EXP]')
            self.eos_token_id = self.tokenizer.convert_tokens_to_ids('</s>')
            self.pad_token_id = self.tokenizer.convert_tokens_to_ids('<pad>')
            print("After adding additional_special_tokens")
            print(self.tokenizer.all_special_tokens)
            print(self.tokenizer.all_special_ids)
            print(f"exp_id : {self.exp_token_id}")
            print(f"eos_id : {self.eos_token_id}")
            print(f"pad_id : {self.pad_token_id}")
            self.tokenizer_length = len(self.tokenizer)
            print(f"len(tokenizer) : {self.tokenizer_length}")

            return self.tokenizer

        elif self.TOKENIZER_NAME in ['facebook/bart-base']:
            self.initialization_flag = 1
            self.tokenizer = AutoTokenizer.from_pretrained(self.TOKENIZER_NAME)
            print(f"tokenizer : AutoTokenizer.from_pretrained({self.TOKENIZER_NAME})")
            print("Before adding additional_special_tokens")
            print(self.tokenizer.all_special_tokens)
            print(self.tokenizer.all_special_ids)
            special_tokens_dict = {
            'additional_special_tokens': ['[EXP]'],
            }
            self.eos_token = '</s>'
            self.pad_token = '<pad>'
            self.tokenizer.add_special_tokens(special_tokens_dict)
            self.exp_token_id = self.tokenizer.convert_tokens_to_ids('[EXP]')
            self.eos_token_id = self.tokenizer.convert_tokens_to_ids('</s>')
            self.pad_token_id = self.tokenizer.convert_tokens_to_ids('<pad>')
            print("After adding additional_special_tokens")
            print(self.tokenizer.all_special_tokens)
            print(self.tokenizer.all_special_ids)
            print(f"exp_id : {self.exp_token_id}")
            print(f"eos_id : {self.eos_token_id}")
            print(f"pad_id : {self.pad_token_id}")
            self.tokenizer_length = len(self.tokenizer)
            print(f"len(tokenizer) : {self.tokenizer_length}")

            return self.tokenizer
        else:
            raise ValueError('Wrong Tokenizer Type : you have to indicate tokenizer type out of the folloing list.\n ["gpt2", "gpt2-medium", "t5-small", "t5-base", "bart-base"]\n')

    def get_tokenizer_info(self):
        if self.initialization_flag == 0:
            print("there is no information. you need to call get_tokenizer() first.")
        else:
            print("---------- tokenizer information ----------")
            print(f"self.TOKENIZER_NAME : {self.TOKENIZER_NAME}")
            print(f"self.tokenizer : {self.tokenizer}")
            print(f"self.tokenizer_length : {self.tokenizer_length}")
            print(f"self.additional_special_tokens : {self.additional_special_tokens}")
            print(f"self.eos_token : {self.eos_token}")
            print(f"self.eos_token_id : {self.eos_token_id}")
            print(f"self.pad_token : {self.pad_token}")
            print(f"self.pad_token_id : {self.pad_token_id}")
            print(f"self.exp_token : {self.exp_token}")
            print(f"self.exp_token_id : {self.exp_token_id}")


# class Prepare_Modelを定義する
- 今のところ、モデルごとに操作の大きな違いはなさそう
- BARTに関して,文の頭に`<s>`が付く

In [ ]:
from transformers import GPT2LMHeadModel, T5ForConditionalGeneration, BartForConditionalGeneration, AutoConfig
import torch
from torch import nn

class Prepare_Model():

    def __init__(self, config:dict, prepare_tokenizer):
        self.MODEL_NAME = config['MODEL_NAME']
        self.TOKENIZER_LENGTH = prepare_tokenizer.tokenizer_length
        self.additional_model_config = None
        self.prepare_tokenizer = prepare_tokenizer
        self.model = None
        self.initialization_flag = 0


    def get_model(self):

        if self.MODEL_NAME in ['gpt2', 'gpt2-medium']:

            if self.prepare_tokenizer is None:
                raise ValueError('prepare_tokenizer is None and this is not good for gpt2!')
            self.initialization_flag = 1
            self.additional_model_config = AutoConfig.from_pretrained(self.MODEL_NAME, eos_token_id=self.prepare_tokenizer.eos_token_id, pad_token_id=self.prepare_tokenizer.pad_token_id, additional_special_tokens_id = self.prepare_tokenizer.exp_token_id, output_hidden_states=False)
            self.model = GPT2LMHeadModel.from_pretrained(self.MODEL_NAME, config=self.additional_model_config)
            self.model.resize_token_embeddings(self.TOKENIZER_LENGTH)
            self.lm_head = self.model.lm_head
            new_weights = torch.cat([self.lm_head.weight[:-1, :], torch.zeros(1, self.lm_head.weight.shape[1]) -10000])
            self.model.lm_head.weight = torch.nn.Parameter(new_weights)

            return self.model

        elif self.MODEL_NAME in ['t5-small','t5-base']:

            self.initialization_flag = 1
            self.model = T5ForConditionalGeneration.from_pretrained(self.MODEL_NAME)
            self.model.resize_token_embeddings(self.TOKENIZER_LENGTH)
            # not sure the following 2 lines are actually necessary
            new_weights = torch.cat([self.model.lm_head.weight[:-1, :], torch.zeros(1, self.model.lm_head.weight.shape[1]) -10000])
            self.model.lm_head.weight = torch.nn.Parameter(new_weights)
            return self.model

        elif self.MODEL_NAME in ['facebook/bart-base']:

            self.initialization_flag = 1
            self.model = BartForConditionalGeneration.from_pretrained(self.MODEL_NAME)
            self.model.resize_token_embeddings(self.TOKENIZER_LENGTH)
            # not sure the following 2 lines are actually necessary
            new_weights = torch.cat([self.model.lm_head.weight[:-1, :], torch.zeros(1, self.model.lm_head.weight.shape[1]) -10000])
            self.model.lm_head.weight = torch.nn.Parameter(new_weights)
            return self.model
        else:
            raise ValueError('Wrong Model Type : you have to indicate model type out of the folloing list.\n ["gpt2", "gpt2-medium", "t5-small", "t5-base", "bart-base"]\n')


    def get_model_info(self):
        if self.initialization_flag == 0:
            print("there is no information. you need to call get_model() first.")
        else:
            print("---------- model information ----------")
            print(f"self.MODEL_NAME : {self.MODEL_NAME}")


In [ ]:
# my_prepare_tokenizer = Prepare_Tokenizer(bart_base_config)
# my_prepare_tokenizer.get_tokenizer_info()
# my_tokenizer = my_prepare_tokenizer.get_tokenizer()
# my_prepare_tokenizer.get_tokenizer_info()
# my_prepare_model = Prepare_Model(config=bart_base_config, prepare_tokenizer=my_prepare_tokenizer)
# my_prepare_model.get_model_info()
# my_model = my_prepare_model.get_model()
# my_prepare_model.get_model_info()

# Trainerを定義する


In [ ]:
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

random_seed = gpt2_small_config["RANDOM_SEED"]
pl.seed_everything(random_seed)

print("train_data_path : " + gpt2_small_config["TRAINING_DATA_PATH"])
print("test_data_path : " + gpt2_small_config["TEST_DATA_PATH"])
print("val_data_path : " + gpt2_small_config["VALIDATION_DATA_PATH"])
train_data = read_tsv(gpt2_small_config["TRAINING_DATA_PATH"])
test_data = read_tsv(gpt2_small_config["TEST_DATA_PATH"])
val_data = read_tsv(gpt2_small_config["VALIDATION_DATA_PATH"])


data_module =  LIAR_PLUS_DataModule_For_CEG(
    gpt2_small_config,
    train_data,
    test_data,
    val_data
    )

total_training_steps, warmup_steps = calculate_warmup_steps(train_data, gpt2_small_config["TRAINING_EPOCHS"],
                                                            gpt2_small_config["TRAINING_BATCH_SIZE"])

print(f"total_training_steps : {total_training_steps} , warmup_steps : {warmup_steps} ")

model = CEG(
    config_dict=gpt2_small_config,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    n_classes=2
    )

model
checkpoint_callback = build_checkpoint_callback(gpt2_small_config)
logger = TensorBoardLogger("lightning_logs", name="CEG")
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=gpt2_small_config['TRAINING_EPOCHS'])

# trainer.fit()

In [ ]:
trainer.fit(model, datamodule=data_module)

# trainer.test()

In [ ]:
trainer.test(datamodule=data_module)

# 結果を確認する

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

# GPT2

## 推論

In [ ]:
## modelのロード
my_model = CEG.load_from_checkpoint(gpt2_small_config['SAVED_MODEL_PATH']+'/QTag-epoch=20-val_loss=7.58.ckpt')
my_model.model.save_pretrained('./model_transformers/gpt2_small')
my_model = GPT2LMHeadModel.from_pretrained('./model_tranformers/gpt2_small')

In [ ]:
## tokenzierの準備
my_prepare_tokenizer = Prepare_Tokenizer(gpt2_small_config)
my_prepare_tokenizer.get_tokenizer_info()
my_tokenizer = my_prepare_tokenizer.get_tokenizer()
my_prepare_tokenizer.get_tokenizer_info()

# データの準備
inputs = my_tokenizer(["Statement: Building a wall on the U.S.-Mexico border will take literally years. Metadata: immigration rick-perry Governor Texas republican Radio interview[EXP]"], return_tensors="pt")
input_length = len(inputs["input_ids"][0])
print(f"inputs_len : {input_length}")

outputs = my_model.generate(
    **inputs,
    max_new_tokens=5,
    num_beams=4,
    num_return_sequences=1,
    return_dict_in_generate=True,
    output_scores=True,
)
print(f"outputs.sequences : {outputs.sequences}")
generated_tokens = outputs.sequences[:, input_length:]
print(f"generated_tokens : {generated_tokens}")

tokens_list = outputs.sequences[0]
print(tokens_list)
full_text = my_tokenizer.decode(tokens_list)
generated = my_tokenizer.decode(generated_tokens[0])

print(f"full text : {full_text}\n")
print(f"generated text: {generated}\n")
print(f"len(generated) : {len(my_tokenizer.tokenize(generated))}\n")

In [ ]:
from transformers import GPT2Tokenizer, AutoConfig, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(["Today is"], return_tensors="pt")
input_length = len(inputs["input_ids"][0])
print(f"inputs_len : {input_length}")

# greedy_search
# outputs = model.generate(**inputs, max_new_tokens=15, return_dict_in_generate=True, output_scores=True)
# beam_search
outputs = model.generate(
    **inputs,
    max_new_tokens=5,
    num_beams=4,
    num_return_sequences=1,
    return_dict_in_generate=True,
    output_scores=True,
)
print(f"outputs.sequences : {outputs.sequences}")
generated_tokens = outputs.sequences[:, input_length:]
print(f"generated_tokens : {generated_tokens}")

tokens_list = outputs.sequences[0]
print(tokens_list)
full_text = tokenizer.decode(tokens_list)
generated = tokenizer.decode(generated_tokens[0])

print(f"full text : {full_text}\n")
print(f"generated text: {generated}\n")
print(f"len(generated) : {len(tokenizer.tokenize(generated))}\n")

# T5デモ
- `<extra_id_0>`や`</s>`が出力に含まれてしまっている
- lm_head.weightsを初期化してもあまり効果が無い
- https://github.com/huggingface/transformers/blob/main/src/transformers/models/t5/modeling_t5.py
- https://www.kaggle.com/code/kreeshrajani/fine-tune-t5-for-conversational-model
- https://huggingface.co/docs/transformers/model_doc/t5#transformers.T5Tokenizer.sp_model_kwargs

## 公式サイトより

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# training
input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids
labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="pt").input_ids
outputs = model(input_ids=input_ids, labels=labels)
loss = outputs.loss
logits = outputs.logits

# inference
input_ids = tokenizer(
    "summarize: studies have shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# studies have shown that owning a dog is good for you.

## fine-tuning

In [ ]:
from transformers import AutoConfig, AutoTokenizer, T5ForConditionalGeneration
import torch
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("t5-small")



print("Before")
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
special_tokens_dict = {
    'additional_special_tokens': ['[EXP]']
    }

tokenizer.add_special_tokens(special_tokens_dict)
exp_id = tokenizer.convert_tokens_to_ids('[EXP]')
eos_id = tokenizer.convert_tokens_to_ids('</s>')
pad_id = tokenizer.convert_tokens_to_ids('<pad>')
print("After")
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
print(f"exp_id : {exp_id}")
print(f"eos_id : {eos_id}")
print(f"pad_id : {pad_id}\n")
print(f"len(tokenizer) : {len(tokenizer)}")

model = T5ForConditionalGeneration.from_pretrained("t5-small")
print(f"before : {model.lm_head.weight.shape}")
print(f"before resizing : {model.lm_head.weight}")

model.resize_token_embeddings(len(tokenizer))

print("after : ", model.lm_head.weight.shape)
print(f"after resizing : {model.lm_head.weight.shape}")
print(f"before changing weights : {model.lm_head.weight.shape}")
print(f"focus : {model.lm_head.weight[:-1, :]}")
print(f"focus.shape : {model.lm_head.weight[:-1, :].shape}")
new_weights = torch.cat([model.lm_head.weight[:-1, :], torch.zeros(1, model.lm_head.weight.shape[1]) -10000])
model.lm_head.weight = torch.nn.Parameter(new_weights)
print(f"after changing weights : {model.lm_head.weight}")
print(f"new_weights.shape : {model.lm_head.weight.shape}")



In [ ]:
input_text = 'Statement: When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration. Metadata: energy,history,job-accomplishments scott-surovell State delegate Virginia democrat a floor speech.'+ tokenizer.convert_ids_to_tokens(exp_id)
justification_text = 'Surovell said the decline of coal "started when natural gas took off  That started to begin in President (George W. ) Bushs administration. "No doubt, natural gas has been gaining ground on coal in generating electricity. The trend started in the 1990s but clearly gained speed during the Bush administration when the production of natural gas -- a competitor of coal -- picked up. But analysts give little credit or blame to Bush for that trend. They note that other factors, such as technological innovation, entrepreneurship and policies of previous administrations, had more to do with laying the groundwork for the natural gas boom.'
label_text = input_text  + justification_text + tokenizer.convert_ids_to_tokens(eos_id)
print(f"tokenized input_text : {tokenizer.tokenize(input_text)}\n")
print(f"tokenized label_text : {tokenizer.tokenize(label_text)}")
input_text_length = len(tokenizer.tokenize(input_text))
justification_text_length = len(tokenizer.tokenize(justification_text))
label_text_length = len(tokenizer.tokenize(label_text))
print(f"input_text_length : {input_text_length}")

input_encoding = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=label_text_length,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)

print(f"input_encoding : {input_encoding}\n")
decoded_input_encoding = tokenizer.decode(input_encoding["input_ids"][0])
# print(f"decoded input_encoding : {decoded_input_encoding}\n")

label_encoding = tokenizer.encode_plus(
    label_text,
    add_special_tokens=True,
    max_length=label_text_length,
    return_token_type_ids=False,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)

print(f"label_encoding : {label_encoding}")
# print(f"label_encoding['input_ids'][0] : {label_encoding['input_ids'][0]}")
decoded_label_encoding = tokenizer.decode(label_encoding["input_ids"][0])
# print(f"decoded label_encoding : {decoded_label_encoding}")
print(f"input prompt : {decoded_input_encoding}")
print(f"label : {decoded_label_encoding}")

labels = label_encoding["input_ids"]
print(f"labels before -100 masking : {labels}")
# decoded_labels = tokenizer.decode(labels)
# print(f"decoded labels : {decoded_labels}")
labels[:, :input_text_length] = -100 # cross_entropy_ignore_index
labels[:, label_text_length:] = -100
print(f"labels after -100 masking : {labels}")
print(f"labels.shape : {labels.shape}")

In [ ]:
outputs = model(input_ids=input_encoding["input_ids"], attention_mask=input_encoding["attention_mask"], labels=labels)

print(f"model : {model}")
print(f"type(model) : {type(model)}")
print(f"type(outputs) : {type(outputs)}")

logits = outputs.logits
loss = outputs.loss
print(f"logits : {logits}")
print(f"logtis.size() : {logits.size()}")
print(f"loss : {loss}")

m = nn.Softmax(dim=1)
output = m(logits)
print(f"output : {output}")
print(f"output.size() : {output.size()}")
arg = torch.argmax(output, dim=2)
print(f"argmax(ouput) : {arg}") # --> tensor
print(f"argmax(ouput).size() : {arg.size()}")
print(f"argmax(output).dtype() : {arg.dtype}")
print(arg)
generated = tokenizer.decode(arg[0])
print(f"generated : {generated}")
print(f"len(generated) : {len(tokenizer.tokenize(generated))}")



## 推論

In [ ]:
from transformers import AutoConfig, AutoTokenizer, T5ForConditionalGeneration
import torch
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(["summarize: Today is"], return_tensors="pt")
input_length = len(inputs["input_ids"][0])
print(f"inputs_len : {input_length}")

# greedy_search
outputs = model.generate(**inputs, max_new_tokens=15, return_dict_in_generate=True, output_scores=True)
# beam_search
# outputs = model.generate(
#     **inputs,
#     max_new_tokens=5,
#     num_beams=4,
#     num_return_sequences=1,
#     return_dict_in_generate=True,
#     output_scores=True,
# )
print(f"outputs.sequences : {outputs.sequences}")
generated_tokens = outputs.sequences[:, input_length:]
print(f"generated_tokens : {generated_tokens}")

tokens_list = outputs.sequences[0]
print(tokens_list)
full_text = tokenizer.decode(tokens_list)
generated = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

print(f"full text : {full_text}\n")
print(f"generated text: {generated}\n")
print(f"len(generated) : {len(tokenizer.tokenize(generated))}\n")

# BART
- add_special_token=Trueにしたとき、先頭に`<s>`が追加されるため-100のマスキング時にinput_length+1

In [ ]:
from transformers import AutoConfig, AutoTokenizer, BartForConditionalGeneration
import torch
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

print("Before")
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
special_tokens_dict = {
    'additional_special_tokens': ['[EXP]']
    }

tokenizer.add_special_tokens(special_tokens_dict)
exp_id = tokenizer.convert_tokens_to_ids('[EXP]')
eos_id = tokenizer.convert_tokens_to_ids('</s>')
pad_id = tokenizer.convert_tokens_to_ids('<pad>')
print("After")
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
print(f"exp_id : {exp_id}")
print(f"eos_id : {eos_id}")
print(f"pad_id : {pad_id}\n")
print(f"len(tokenizer) : {len(tokenizer)}")

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
# print(f"before : {model.lm_head.weight.shape}")
# print(f"before resizing : {model.lm_head.weight}")

model.resize_token_embeddings(len(tokenizer))

# print("after : ", model.lm_head.weight.shape)
# print(f"after resizing : {model.lm_head.weight.shape}")
# print(f"before changing weights : {model.lm_head.weight.shape}")
# print(f"focus : {model.lm_head.weight[:-1, :]}")
# print(f"focus.shape : {model.lm_head.weight[:-1, :].shape}")
# new_weights = torch.cat([model.lm_head.weight[:-1, :], torch.zeros(1, model.lm_head.weight.shape[1]) -10000])
# model.lm_head.weight = torch.nn.Parameter(new_weights)
# print(f"after changing weights : {model.lm_head.weight}")
# print(f"new_weights.shape : {model.lm_head.weight.shape}")



In [ ]:
input_text = 'Statement: When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration. Metadata: energy,history,job-accomplishments scott-surovell State delegate Virginia democrat a floor speech.'+ tokenizer.convert_ids_to_tokens(exp_id)
justification_text = 'Surovell said the decline of coal "started when natural gas took off  That started to begin in President (George W. ) Bushs administration. "No doubt, natural gas has been gaining ground on coal in generating electricity. The trend started in the 1990s but clearly gained speed during the Bush administration when the production of natural gas -- a competitor of coal -- picked up. But analysts give little credit or blame to Bush for that trend. They note that other factors, such as technological innovation, entrepreneurship and policies of previous administrations, had more to do with laying the groundwork for the natural gas boom.'
label_text = input_text  + justification_text + tokenizer.convert_ids_to_tokens(eos_id)
print(f"tokenized input_text : {tokenizer.tokenize(input_text)}\n")
print(f"tokenized label_text : {tokenizer.tokenize(label_text)}")
input_text_length = len(tokenizer.tokenize(input_text))
justification_text_length = len(tokenizer.tokenize(justification_text))
label_text_length = len(tokenizer.tokenize(label_text))
print(f"input_text_length : {input_text_length}")

input_encoding = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=label_text_length,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)

print(f"input_encoding : {input_encoding}\n")
decoded_input_encoding = tokenizer.decode(input_encoding["input_ids"][0])
# print(f"decoded input_encoding : {decoded_input_encoding}\n")

label_encoding = tokenizer.encode_plus(
    label_text,
    add_special_tokens=True,
    max_length=label_text_length,
    return_token_type_ids=False,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)

print(f"label_encoding : {label_encoding}")
# print(f"label_encoding['input_ids'][0] : {label_encoding['input_ids'][0]}")
decoded_label_encoding = tokenizer.decode(label_encoding["input_ids"][0])
# print(f"decoded label_encoding : {decoded_label_encoding}")
print(f"input prompt : {decoded_input_encoding}")
print(f"label : {decoded_label_encoding}")

labels = label_encoding["input_ids"]
print(f"labels before -100 masking : {labels}")
# decoded_labels = tokenizer.decode(labels)
# print(f"decoded labels : {decoded_labels}")
labels[:, :input_text_length+1] = -100 # cross_entropy_ignore_index
labels[:, label_text_length:] = -100
print(f"labels after -100 masking : {labels}")
print(f"labels.shape : {labels.shape}")

In [ ]:
outputs = model(input_ids=input_encoding["input_ids"], attention_mask=input_encoding["attention_mask"], labels=labels)

print(f"model : {model}")
print(f"type(model) : {type(model)}")
print(f"type(outputs) : {type(outputs)}")

logits = outputs.logits
loss = outputs.loss
print(f"logits : {logits}")
print(f"logtis.size() : {logits.size()}")
print(f"loss : {loss}")

m = nn.Softmax(dim=1)
output = m(logits)
print(f"output : {output}")
print(f"output.size() : {output.size()}")
arg = torch.argmax(output, dim=2)
print(f"argmax(ouput) : {arg}") # --> tensor
print(f"argmax(ouput).size() : {arg.size()}")
print(f"argmax(output).dtype() : {arg.dtype}")
print(arg)
generated = tokenizer.decode(arg[0])
print(f"generated : {generated}")
print(f"len(generated) : {len(tokenizer.tokenize(generated))}")



## 推論